A deep resnet trained on the pokemon image dataset for type prediction. The preprocessing was done according to https://www.kaggle.com/bishetheanswer/pokemon-image-dataset-ready-for-type-prediction.

In [ ]:
%config Completer.use_jedi = False

In [ ]:
import tensorflow as tf
import os
import matplotlib
import pandas as pd
import numpy as np
import shutil

from tensorflow.keras import layers
from PIL import Image
import matplotlib.pyplot as plt
seed = 27912
np.random.seed(seed)

In [ ]:
data = pd.read_csv("downloaded_datasets/pokemon/pokemon.csv")
data.info

In [ ]:
types = data.Type1.unique()
types

In [ ]:
os.mkdir("downloaded_datasets/pokemon/newData_pokemon")

for t in types:
    os.mkdir("downloaded_datasets/pokemon/newData_pokemon/{}".format(t))

In [ ]:
images_dir = "downloaded_datasets/pokemon/images/images/"

In [ ]:
for t in types:
    aux_type = data[data.Type1.eq(t)]
    for pokemon in aux_type.Name:
        for filename in os.listdir(images_dir):
            original_path = "{}{}".format(images_dir, filename)
            # pokemon name with extension
            extension = os.path.basename(original_path)
            # directory without extension
            poke_dir = os.path.splitext(original_path)[0]
            # only pokemon name
            poke_name = os.path.basename(poke_dir)
            if(pokemon == poke_name):
                target_path = "downloaded_datasets/pokemon/newData_pokemon/{}/{}".format(t, extension)
                shutil.copyfile(original_path, target_path)

In [ ]:
fill_color = (255, 255, 255)
new_images_dir = "downloaded_datasets/pokemon/newData_pokemon/"

for t in types:
    for filename in os.listdir(new_images_dir):
        type_dir = "{}{}/".format(new_images_dir, filename)
        for pokemon in os.listdir(type_dir):
            full_path = "{}/{}".format(type_dir, pokemon)
            file_dir, file_extension = os.path.splitext(full_path)
            if file_extension == ".png":
                im = Image.open(full_path)
                im = im.convert("RGBA")
                if im.mode in ('RGBA', 'LA'):
                    bg = Image.new(im.mode[:-1], im.size, fill_color)
                    bg.paste(im, im.split()[-1])  # omit transparency
                    bg.save("{}.jpg".format(file_dir))
                    os.remove(full_path)

In [ ]:
dir_name = "downloaded_datasets/pokemon/newData_pokemon/"
output_filename = "pokemon_data"

shutil.make_archive(output_filename, 'zip', dir_name)

In [ ]:
img_height = 120
img_width = 120
new_images_dir = "downloaded_datasets/pokemon/newData_pokemon/"

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    new_images_dir,
    validation_split=0.2,
    subset="training",
    image_size = (img_height, img_width),
    seed=seed,
    label_mode='categorical',
)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    new_images_dir,
    validation_split=0.2,
    subset="validation",
    image_size = (img_height,img_width),
    seed=seed,
    label_mode='categorical',
)

In [ ]:
class_names = train_ds.class_names
print(class_names)
print(len(class_names))

In [ ]:
idx_to_class_names = {}
for i in range(18):
    idx_to_class_names[i] = class_names[i]

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(idx_to_class_names[np.argmax(labels[i])])
        plt.axis("off")

In [ ]:
for image, labels in train_ds:
    print(image.shape)
    print(labels.shape)
    break

In [ ]:
def identity_block(X, filters):
    F1, F2, F3 = filters
    X_shortcut = X
    X = layers.Conv2D(F1, (1,1))(X)
    X = layers.BatchNormalization()(X)
    X = layers.Activation("relu")(X)
    X = layers.Conv2D(F2, (3,3), padding="same")(X)
    X = layers.BatchNormalization()(X)
    X = layers.Activation("relu")(X)
    X = layers.Conv2D(F3, (1,1))(X)
    X = layers.BatchNormalization()(X)
    X = layers.Add()([X, X_shortcut])
    X = layers.Activation("relu")(X)
    return X

def convolutional_block(X, filters, s=2):
    F1, F2, F3 = filters
    X_shortcut = X
    X = layers.Conv2D(F1, (1,1), strides=(s,s))(X)
    X = layers.BatchNormalization()(X)
    X = layers.Activation("relu")(X)
    X = layers.Conv2D(F2, (3,3), padding="same")(X)
    X = layers.BatchNormalization()(X)
    X = layers.Activation("relu")(X)
    X = layers.Conv2D(F3, (1,1))(X)
    X = layers.BatchNormalization()(X)
    X_shortcut = layers.Conv2D(F3, (1,1),strides=(s,s))(X_shortcut)
    X_shortcut = layers.BatchNormalization()(X_shortcut)
    X = layers.Add()([X, X_shortcut])
    X = layers.Activation("relu")(X)
    return X

In [ ]:
def resnet(input_shape):
    X_input = tf.keras.Input(input_shape)
    X = layers.Conv2D(64, (3,3), padding="same")(X_input)
    X = layers.BatchNormalization()(X)
    X = layers.MaxPooling2D()(X)
    X = convolutional_block(X, filters=[64,64,256], s=1)
    X = identity_block(X, filters=[64,64,256])
    X = identity_block(X, filters=[64,64,256])
    X = identity_block(X, filters=[64,64,256])
    X = convolutional_block(X, filters=[128,128,512])
    X = identity_block(X, filters=[128,128,512])
    X = identity_block(X, filters=[128,128,512])
    X = identity_block(X, filters=[128,128,512])
    X = convolutional_block(X, filters=[256,256,1024])
    X = identity_block(X, filters=[256,256,1024])
    X = identity_block(X, filters=[256,256,1024])
    X = identity_block(X, filters=[256,256,1024])
    X = convolutional_block(X, filters=[512,512,2048])
    X = identity_block(X, filters=[512,512,2048])
    X = identity_block(X, filters=[512,512,2048])
    X = identity_block(X, filters=[512,512,2048])
    X = layers.AveragePooling2D()(X)
    X = layers.Flatten()(X)
    X = layers.Dense(18, activation="softmax")(X)
    model = tf.keras.models.Model(inputs = X_input, outputs=X)
    return model

In [ ]:
model = resnet(input_shape=(120,120,3))

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, to_file="models/deep_resnet.png")

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(train_ds, epochs=30, validation_data=val_ds)